In [ ]:
from src.utils import *

In [2]:
# import wav data to keras dataset
import numpy as np
from keras.utils import audio_dataset_from_directory

# Load audio dataset from directory
train_dataset, validation_dataset = audio_dataset_from_directory(
    directory='data/path/train',
    validation_split=0.2,
    subset="both",
    labels="inferred",
    label_mode="categorical",
    seed=1337,
    batch_size=64,
    output_sequence_length=16000, 
)


Found 4752 files belonging to 2 classes.
Using 3802 files for training.
Using 950 files for validation.


2024-04-22 13:48:54.971207: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 13:48:55.091680: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 13:48:55.091743: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 13:48:55.094383: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 13:48:55.094436: I external/local_xla/xla/stream_executor

In [3]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
from tensorflow.keras.optimizers import Adam
from transformers import AutoFeatureExtractor, TFWav2Vec2ForSequenceClassification

# Load pre-trained Wav2Vec2 model and processor using AutoFeatureExtractor
model_name = "facebook/wav2vec2-base-960h"
tokenizer = AutoFeatureExtractor.from_pretrained(model_name)

# Load pre-trained Wav2Vec2 model
model = TFWav2Vec2ForSequenceClassification.from_pretrained(model_name)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]



TFWav2Vec2ForSequenceClassification has backpropagation operations that are NOT supported on CPU. If you wish to train/fine-tune this model, you need a GPU or a TPU
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFWav2Vec2ForSequenceClassification: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing TFWav2Vec2ForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFWav2Vec2ForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFWav2Vec2ForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['wav2vec2.masked_spec_embed', 'pr

In [4]:
import tensorflow as tf
from transformers import AutoFeatureExtractor
from tqdm.notebook import tqdm

# Load Wav2Vec tokenizer
model_name = "facebook/wav2vec2-base-960h"
tokenizer = AutoFeatureExtractor.from_pretrained(model_name)
print("test")

# Define a function to tokenize audio data
def tokenize_dataset(dataset):
    tokenized_batches = []
    for batch in tqdm(dataset, total=len(dataset)):
        
        audio_batch, labels = batch
        # print(audio_batch)
        inputs = tokenizer(audio_batch, return_tensors="tf", padding=True, verbose=False, sampling_rate=16000)
        tokenized_batches.append((inputs.input_values, labels))
    return tokenized_batches

# Tokenize the train dataset
tokenized_train_dataset = tokenize_dataset(train_dataset)

# Tokenize the validation dataset
tokenized_validation_dataset = tokenize_dataset(validation_dataset)


test


  0%|          | 0/60 [00:00<?, ?it/s]

2024-04-22 13:49:51.359025: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


  0%|          | 0/15 [00:00<?, ?it/s]

2024-04-22 13:50:03.599788: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


import tensorflow as tf

# Convert tokenized batches into a generator function
def batch_generator(tokenized_batches):
    for input_values, labels in tokenized_batches:
        yield input_values, labels

# Create Keras batch datasets for train and validation datasets
keras_train_dataset = tf.data.Dataset.from_generator(
    generator=lambda: batch_generator(tokenized_train_dataset),
    output_signature=(
        tf.TensorSpec(shape=(1, 32, 16000, 1), dtype=tf.float32),  # Input values
        tf.TensorSpec(shape=(32, 30), dtype=tf.int32)           # Labels
    )
)

keras_validation_dataset = tf.data.Dataset.from_generator(
    generator=lambda: batch_generator(tokenized_validation_dataset),
    output_signature=(
        tf.TensorSpec(shape=(1, 32, 16000, 1), dtype=tf.float32),  # Input values
        tf.TensorSpec(shape=(32, 30), dtype=tf.int32)           # Labels
    )
)



In [5]:
X_train = tf.concat([i[0][0] for i in tokenized_train_dataset], axis=0)
X_val = tf.concat([i[0][0] for i in tokenized_validation_dataset], axis=0)

In [6]:
y_train = tf.concat([i[1] for i in tokenized_train_dataset], axis=0)
y_val = tf.concat([i[1] for i in tokenized_validation_dataset], axis=0)

In [7]:
X_train = X_train.numpy()
X_val = X_val.numpy()

In [8]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1])
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1])

In [12]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Dropout, Dense
from tensorflow.keras.layers import Bidirectional, LSTM, TimeDistributed


def get_SR_Model(num_classes: int):
    X_input = Input(shape=(16000, 1))
    X = Conv1D(filters=256,kernel_size=15,strides=4)(X_input)
    X = BatchNormalization()(X)
    X = Dropout(0.2)(X)
    X = Conv1D(filters=512,kernel_size=15,strides=4)(X_input)
    X = BatchNormalization()(X)
    X = Dropout(0.2)(X)
    X = LSTM(units=512, return_sequences=True)(X)
    X = LSTM(units=512, return_sequences=False)(X)
    X = Dense(num_classes, activation='softmax')(X)
    return Model(inputs=[X_input], outputs=[X])

model = get_SR_Model(2)



In [20]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from transformers import TFWav2Vec2Model


def get_wav2vec_classifier(num_classes: int):
    # Load pre-trained Wav2Vec model
    model_name = "facebook/wav2vec2-base-960h"
    wav2vec_model = TFWav2Vec2Model.from_pretrained(model_name)

    # Freeze the pre-trained Wav2Vec model
    wav2vec_model.trainable = False

    # Define input layer for tokenized input
    tokenized_input = Input(shape=(None,), dtype=tf.int32)

    # Convert input to TensorFlow tensor
    input_tensor = tf.convert_to_tensor(tokenized_input)

    # Get hidden states from pre-trained Wav2Vec model
    hidden_states = wav2vec_model(input_values=input_tensor).last_hidden_state

    # Add classification layers
    x = Dense(512, activation='relu')(hidden_states[:, 0, :])  # Take the first token's representation
    x = Dropout(0.2)(x)
    x = Dense(num_classes, activation='softmax')(x)

    # Define model
    model = Model(inputs=tokenized_input, outputs=x)

    return model


# Example usage:
model = get_wav2vec_classifier(num_classes=2)




TFWav2Vec2Model has backpropagation operations that are NOT supported on CPU. If you wish to train/fine-tune this model, you need a GPU or a TPU
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFWav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing TFWav2Vec2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFWav2Vec2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFWav2Vec2Model were not initialized from the PyTorch model and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inf

ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [13]:
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                loss=losses.CategoricalCrossentropy(),
                metrics=[metrics.CategoricalAccuracy()])

In [14]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64)

Epoch 1/10


2024-04-22 13:50:21.515564: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8900


60/60 ━━━━━━━━━━━━━━━━━━━━ 46s 722ms/step - categorical_accuracy: 0.5064 - loss: 0.7088 - val_categorical_accuracy: 0.5042 - val_loss: 0.6939
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 44s 732ms/step - categorical_accuracy: 0.5249 - loss: 0.6965 - val_categorical_accuracy: 0.5042 - val_loss: 0.6937
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 44s 734ms/step - categorical_accuracy: 0.5227 - loss: 0.6879 - val_categorical_accuracy: 0.5895 - val_loss: 0.5486
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 43s 719ms/step - categorical_accuracy: 0.5716 - loss: 0.6828 - val_categorical_accuracy: 0.5000 - val_loss: 1.3640
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 43s 718ms/step - categorical_accuracy: 0.4986 - loss: 0.7566 - val_categorical_accuracy: 0.4989 - val_loss: 0.6943
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 44s 727ms/step - categorical_accuracy: 0.5126 - loss: 0.6968 - val_categorical_accuracy: 0.4926 - val_loss: 0.6986
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 44s 735ms/step - categorical_accuracy: 0.4955 - loss: 0

KeyboardInterrupt: 